# OB 3조
# 텍스트 분석 - Sentiment140 dataset with 1.6 million tweets

In [ ]:
# 필요한 모듈 불러오기
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/dataset/tweet.csv',encoding='ISO-8859-1',names=['target','id','date','flag','user','text'],index_col=False)
data.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# 1. 연습용 데이터 분리(500개)

In [ ]:
# import random
# from sklearn.model_selection import train_test_split

# #target == 0 에서 250개
# idx1=random.sample(range(0,800000),250)
# df_0=data.iloc[idx1]

# #target == 1 에서 250개
# idx2=random.sample(range(800000,1600000),250)
# df_1=data.iloc[idx2]


# data = pd.concat([df_0,df_1])

# #train/test 분리
# train_0 = data.iloc[0:175]
# test_0 = data.iloc[176:250]
# train_4 = data.iloc[250:425]
# test_4 = data.iloc[425:500]

# train = pd.concat([train_0,train_4])
# test = pd.concat([test_0,test_4])

# train = train.sample(frac=1).reset_index(drop=True)
# test = test.sample(frac=1).reset_index(drop=True)

# y_train = train['target']
# y_test = test['target']



# 본데이터

In [ ]:
import random
from sklearn.model_selection import train_test_split

#target == 0 에서 10000개
idx1=random.sample(range(0,800000),10000)
df_0=data.iloc[idx1]

#target == 1 에서 10000개
idx2=random.sample(range(800000,1600000),10000)
df_1=data.iloc[idx2]


data = pd.concat([df_0,df_1])

#train/test 분리
train_0 = data.iloc[0:7000]
test_0 = data.iloc[7001:10000]
train_4 = data.iloc[10001:17000]
test_4 = data.iloc[17001:20000]

train = pd.concat([train_0,train_4])
test = pd.concat([test_0,test_4])

train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

y_train = train['target']
y_test = test['target']

---
# 2. 전처리

## 0) 전처리에 필요한 함수 정의

0. remove Unicode

In [ ]:
def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', str(text))       
    text = re.sub(r'[^\x00-\x7f]',r'',str(text))
    return text

1. Replace URLs, User Mentions and Hashtags

In [ ]:
def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',str(text))
    text = re.sub(r'#([^\s]+)', r'\1', str(text))
    return text

In [ ]:
def replaceAtUser(text):
    """ Replaces "@user" with "atUser" """
    text = re.sub('@[^\s]+','atUser',str(text))
    return text

In [ ]:
def removeHashtagInFrontOfWord(text):
    """ Removes hastag in front of a word """
    text = re.sub(r'#([^\s]+)', r'\1', str(text))
    return text

2. Replace slang

In [ ]:
""" Creates a dictionary with slangs and their equivalents and replaces them """
with open('slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

In [ ]:
slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])

In [ ]:
def countSlang(text):
    """ Input: a text, Output: how many slang words and a list of found slangs """
    slangCounter = 0
    slangsFound = []
    tokens = nltk.word_tokenize(text)
    for word in tokens:
        if word in slang_words:
            slangsFound.append(word)
            slangCounter += 1
    return slangCounter, slangsFound

3. Replace Contractions

In [ ]:
""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

4. Remove Numbers

In [ ]:
def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

5. Replace Repetitions of Punctuation

In [ ]:
def replaceMultiExclamationMark(text):
    """ Replaces repetitions of exlamation marks """
    text = re.sub(r"(\!)\1+", ' multiExclamation ', str(text))
    return text

def replaceMultiQuestionMark(text):
    """ Replaces repetitions of question marks """
    text = re.sub(r"(\?)\1+", ' multiQuestion ', str(text))
    return text

def replaceMultiStopMark(text):
    """ Replaces repetitions of stop marks """
    text = re.sub(r"(\.)\1+", ' multiStop ', str(text))
    return text

In [ ]:
def countMultiExclamationMarks(text):
    """ Replaces repetitions of exlamation marks """
    return len(re.findall(r"(\!)\1+", str(text)))

def countMultiQuestionMarks(text):
    """ Count repetitions of question marks """
    return len(re.findall(r"(\?)\1+", str(text)))

def countMultiStopMarks(text):
    """ Count repetitions of stop marks """
    return len(re.findall(r"(\.)\1+", str(text)))

6. Replace Negations with Antonyms

In [ ]:
### Replace Negations Begin ###

def replace(word, pos=None):
    """ Creates a set of all antonyms for the word and if there is only one antonym, it returns it """
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
      for lemma in syn.lemmas():
        for antonym in lemma.antonyms():
          antonyms.add(antonym.name())
    if len(antonyms) == 1:
      return antonyms.pop()
    else:
      return None

def replaceNegations(text):
    """ Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym """
    i, l = 0, len(text)
    words = []
    while i < l:
      word = text[i]
      if word == 'not' and i+1 < l:
        ant = replace(text[i+1])
        if ant:
          words.append(ant)
          i += 2
          continue
      words.append(word)
      i += 1
    return words

7. Remove Punctuation
  * python string 내장 메서드 str.translate() 사용

8. Handling Capitalized Words

In [ ]:
def addCapTag(word):
    """ Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_ """
    if(len(re.findall("[A-Z]{3,}", word))):
        word = word.replace('\\', '' )
        transformed = re.sub("[A-Z]{3,}", "ALL_CAPS_"+word, word)
        return transformed
    else:
        return word

9. Lowercase
  * Python string 내장 메서드 str.lower() 사용

10. Remove Stopwords
  * 이후에 나오는 tokenize() 함수 내에 있음

11. Replace Elongated Words

In [ ]:
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

12. Spelling Correction

In [ ]:
### Spell Correction begin ###
""" Spell Correction http://norvig.com/spell-correct.html """
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('corporaForSpellCorrection.txt').read()))

def P(word, N=sum(WORDS.values())): 
    """P robability of `word`. """
    return WORDS[word] / N

def spellCorrection(word): 
    """ Most probable spelling correction for word. """
    return max(candidates(word), key=P)

def candidates(word): 
    """ Generate possible spelling corrections for word. """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    """ The subset of `words` that appear in the dictionary of WORDS. """
    return set(w for w in words if w in WORDS)

def edits1(word):
    """ All edits that are one edit away from `word`. """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    """ All edits that are two edits away from `word`. """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

13. Part of Speech Tagging
  * 이후 전체 실행 셀에 있음
  * Tokenizer()앞 pos_tag())

14. Lemmatizing
  * 이후에 나오는 tokenize() 함수 내에 있음

15. Stemming
  * 이후에 나오는 tokenize() 함수 내에 있음

16. Remove Emoticons

In [ ]:
def countEmoticons(text):
    """ Input: a text, Output: how many emoticons """
    return len(re.findall(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', text))


In [ ]:
def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    return text

17. 추가적으로 필요한 함수들
  * 전체 실행 셀에서 변수 업데이트를 위한 함수 등

In [ ]:
def countAllCaps(text):
    """ Input: a text, Output: how many words are all caps """
    return len(re.findall("[A-Z0-9]{3,}", str(text)))

In [ ]:
def countElongated(text):
    """ Input: a text, Output: how many words are elongated """
    regex = re.compile(r"(.)\1{2}")
    return len([word for word in text.split() if regex.search(word)])

In [ ]:
def addNotTag(text):
	""" Finds "not,never,no" and adds the tag NEG_ to all words that follow until the next punctuation """
	transformed = re.sub(r'\b(?:not|never|no)\b[\w\s]+[^\w\s]', 
       lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), 
       text,
       flags=re.IGNORECASE)
	return transformed

## 1) 전처리 최종 함수 : preprocessing

In [ ]:
""" Copyright 2017, Dimitrios Effrosynidis, All rights reserved. """

from time import time
import numpy as np
import string

#from techniques import *

print("Starting preprocess..\n")

""" Tokenizes a text to its words, removes and replaces some of them """    

def preprocessing(data):
  finalTokens = [] # all tokens
  Tokenslist=[]
  stoplist = stopwords.words('english')
  my_stopwords = "multiexclamation multiquestion multistop url atuser st rd nd th am pm" # my extra stopwords
  stoplist = stoplist + my_stopwords.split()
  allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging
  lemmatizer = WordNetLemmatizer() # set lemmatizer
  stemmer = PorterStemmer() # set stemmer

  def tokenize(text, wordCountBefore, textID, y):
      totalAdjectives = 0
      totalAdverbs = 0
      totalVerbs = 0
      onlyOneSentenceTokens = [] # tokens of one sentence each time

      tokens = nltk.word_tokenize(text)
      
      tokens = replaceNegations(tokens) # Technique 6: finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym
      
      translator = str.maketrans('', '', string.punctuation)
      text = text.translate(translator) # Technique 7: remove punctuation

      tokens = nltk.word_tokenize(text) # it takes a text as an input and provides a list of every token in it
      
  ### NO POS TAGGING BEGIN (If you don't want to use POS Tagging keep this section uncommented) ###
      
  ##    for w in tokens:
  ##
  ##        if (w not in stoplist): # Technique 10: remove stopwords
  ##            final_word = addCapTag(w) # Technique 8: Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_
  ##            final_word = final_word.lower() # Technique 9: lowercases all characters
  ##            final_word = replaceElongated(final_word) # Technique 11: replaces an elongated word with its basic form, unless the word exists in the lexicon
  ##            if len(final_word)>1:
  ##                final_word = spellCorrection(final_word) # Technique 12: correction of spelling errors
  ##            final_word = lemmatizer.lemmatize(final_word) # Technique 14: lemmatizes words
  ##            final_word = stemmer.stem(final_word) # Technique 15: apply stemming to words

  ### NO POS TAGGING END ###


  ### POS TAGGING BEGIN (If you want to exclude words using POS Tagging, keep this section uncommented and comment the above) ###          
              
      tagged = nltk.pos_tag(tokens) # Technique 13: part of speech tagging  
      for w in tagged:

          if (w[1][0] in allowedWordTypes and w[0] not in stoplist):
              final_word = addCapTag(w[0])
              #final_word = final_word.lower()
              final_word = replaceElongated(final_word)
              if len(final_word)>1:
                  final_word = spellCorrection(final_word)
              final_word = lemmatizer.lemmatize(final_word)
              final_word = stemmer.stem(final_word)

  ### POS TAGGING END ###
                  
              onlyOneSentenceTokens.append(final_word)           
      finalTokens.append(onlyOneSentenceTokens)

          
      onlyOneSentence = " ".join(onlyOneSentenceTokens) # form again the sentence from the list of tokens
      #print(onlyOneSentence) # print final sentence

      
      """ Write the preprocessed text to file """
      #with open("result.txt", "a") as result:
      #    result.write(textID+"\t"+y+"\t"+onlyOneSentence+"\n")
          
      return finalTokens

  t0 = time()
  totalSentences = 0
  totalEmoticons = 0
  totalSlangs = 0
  totalSlangsFound = []
  totalElongated = 0
  totalMultiExclamationMarks = 0
  totalMultiQuestionMarks = 0
  totalMultiStopMarks = 0
  totalAllCaps = 0


  for i in range(data.shape[0]):
      totalSentences += 1
      feat = []
      #columns = line.split('\t')
      #columns = [col.strip() for col in columns]
      text=data['text'][i]

      textID = (data['id'][i])
      y = (data['target'][i])

      text = removeUnicode(text) # Technique 0
      #print(text) # print initial text
      wordCountBefore = len(re.findall(r'\w+', text)) # word count of one sentence before preprocess    
      #print("Words before preprocess: ",wordCountBefore,"\n")
      
      text = replaceURL(text) # Technique 1
      text = replaceAtUser(text) # Technique 1
      text = removeHashtagInFrontOfWord(text) # Technique 1

      temp_slangs, temp_slangsFound = countSlang(text)
      totalSlangs += temp_slangs # total slangs for all sentences
      for word in temp_slangsFound:
          totalSlangsFound.append(word) # all the slangs found in all sentences
      
      text = replaceSlang(text) # Technique 2: replaces slang words and abbreviations with their equivalents
      text = replaceContraction(text) # Technique 3: replaces contractions to their equivalents
      text = removeNumbers(text) # Technique 4: remove integers from text

      emoticons = countEmoticons(text) # how many emoticons in this sentence
      totalEmoticons += emoticons
      
      text = removeEmoticons(text) # removes emoticons from text

      
      totalAllCaps += countAllCaps(text)

      totalMultiExclamationMarks += countMultiExclamationMarks(text) # how many repetitions of exlamation marks in this sentence
      totalMultiQuestionMarks += countMultiQuestionMarks(text) # how many repetitions of question marks in this sentence
      totalMultiStopMarks += countMultiStopMarks(text) # how many repetitions of stop marks in this sentence
      
      text = replaceMultiExclamationMark(text) # Technique 5: replaces repetitions of exlamation marks with the tag "multiExclamation"
      text = replaceMultiQuestionMark(text) # Technique 5: replaces repetitions of question marks with the tag "multiQuestion"
      text = replaceMultiStopMark(text) # Technique 5: replaces repetitions of stop marks with the tag "multiStop"

      totalElongated += countElongated(text) # how many elongated words emoticons in this sentence
      
      tokens = tokenize(text, wordCountBefore, textID, y)
      
  """
  전처리에 필요한 런타임 등등 확인하기 위한 부분 => 필요없음

  print("Total sentences: ",totalSentences,"\n")
  print("Total Words before preprocess: ",len(re.findall(r'\w+', f)))
  print("Total Distinct Tokens before preprocess: ",len(set(re.findall(r'\w+', f))))
  print("Average word/sentence before preprocess: ",len(re.findall(r'\w+', f))/totalSentences,"\n")
  print("Total Words after preprocess: ",len(tokens))
  print("Total Distinct Tokens after preprocess: ",len(set(tokens)))
  print("Average word/sentence after preprocess: ",len(tokens)/totalSentences,"\n")


  print("Total run time: ",time() - t0," seconds\n")

  print("Total emoticons: ",totalEmoticons,"\n")
  print("Total slangs: ",totalSlangs,"\n")
  commonSlangs = nltk.FreqDist(totalSlangsFound)
  for (word, count) in commonSlangs.most_common(20): # most common slangs across all texts
      print(word,"\t",count)

  commonSlangs.plot(20, cumulative=False) # plot most common slangs

  print("Total elongated words: ",totalElongated,"\n")
  print("Total multi exclamation marks: ",totalMultiExclamationMarks)
  print("Total multi question marks: ",totalMultiQuestionMarks)
  print("Total multi stop marks: ",totalMultiStopMarks,"\n")
  print("Total all capitalized words: ",totalAllCaps,"\n")

  #print(tokens)
  commonWords = nltk.FreqDist(tokens)
  print("Most common words ")
  print("Word\tCount")
  for (word, count) in commonWords.most_common(100): # most common words across all texts
      print(word,"\t",count)

  commonWords.plot(100, cumulative=False) # plot most common words


  bgm = nltk.collocations.BigramAssocMeasures()
  tgm = nltk.collocations.TrigramAssocMeasures()
  bgm_finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
  tgm_finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
  bgm_finder.apply_freq_filter(5) # bigrams that occur at least 5 times
  print("Most common collocations (bigrams)")
  print(bgm_finder.nbest(bgm.pmi, 50)) # top 50 bigram collocations
  tgm_finder.apply_freq_filter(5) # trigrams that occur at least 5 times
  print("Most common collocations (trigrams)")
  print(tgm_finder.nbest(tgm.pmi, 20)) # top 20 trigrams collocations
  """

  for i in range(len(finalTokens)):
    print(finalTokens[i])
  return finalTokens


Starting preprocess..



## 2) 전처리 함수 실행

In [ ]:
#전처리 함수 실행 후 X_train/X_test에 저장
#Y는 y_train/y_test에 저장되어있음

X_train=preprocessing(train)
X_test=preprocessing(test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['think', 'get', 'job', 'tri', 'financi', 'time', 'start', 'find', 'god', 'pleas', 'tell']
['atus', 'shook', 'who']
['monday', 'fain', 'urlaub', 'concert', 'go', 'multistop', 'envi', 'friend', 'go', 'multistop']
['tire', 'need', 'studi']
['think', 'grass', 'somethingankl', 'leg', 'mummi', 'bald', 'rash', 'adam', 'sticki', 'lot', 'antifung', 'stuff', 'keep', 'bay', 'never', 'go']
['atus', 'voic', 'or', 'promo', 'sampl', 'andquotoh', 'yeah', 'babyandquot', 'voiceov', 'work', 'multistop']
['life', 'seem', 'winfield', 'episod', 'got', 'fight', 'ladi', 'basknrobin', 'sourswetcream', 'of', 'al_caps_refund']
['atus', 'look', 'almost', 'warm']
['atus', 'al_caps_pang', 'multiquest', 'multiexclam', 'al_caps_ahahahahaha', 'meet', 'drink']
['atus', 'sorri', 'miss', 'parti', 'sick', 'bed', 'to', 'buena']
['atus', 'hope', 'great', 'night', 'work', 'sleep', 'well', 'hope', 'chat', 'soon']
['believ', 'teach', 'certif', 'came', 'today']
['to', 'talk', 'too', 'atus']


In [ ]:
#토큰화 리스트로 합치기 : train

for i in range(len(X_train)):
  X_train[i]=" ".join(X_train[i])

In [ ]:
#test

for i in range(len(X_test)):
  X_test[i]=" ".join(X_test[i])

---

In [ ]:
pd.DataFrame(X_train).to_csv('X_train_preprocessing.csv')

In [ ]:
pd.DataFrame(X_test).to_csv('X_test_preprocessing.csv')

In [ ]:
np.where(X_train.isnull())
X_train = X_train.drop(index = [  213,  2025,  4717,  4964,  4974,  5695,  6305,  6614,  7806,
         8909,  9099, 11978, 12850],axis = 0)

y_train = y_train.drop(index = [  213,  2025,  4717,  4964,  4974,  5695,  6305,  6614,  7806,
         8909,  9099, 11978, 12850], axis = 0)

np.where(X_test.isnull())

X_test = X_test.drop(index = [ 973, 1436, 3261, 4536, 4728, 5289, 5813],axis = 0)

y_test = y_test.drop(index = [ 973, 1436, 3261, 4536, 4728, 5289, 5813], axis = 0)

# 3. 피처 벡터화 & 모델링

## 0) Word2Vec 

In [ ]:
#import package

import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score


In [ ]:

model_w2v = Word2Vec(
            X_train,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model_w2v.train(X_train, total_examples= len(X_train), epochs=20)

(28319, 55340)

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
wordvec_arrays_train = np.zeros((len(X_train), 200)) 
for i in range(len(X_train)):
    wordvec_arrays_train[i,:] = word_vector(X_train[i], 200)
X_train_w2v = pd.DataFrame(wordvec_arrays_train)

wordvec_arrays_test = np.zeros((len(X_test), 200)) 
for i in range(len(X_test)):
    wordvec_arrays_test[i,:] = word_vector(X_test[i], 200)
X_test_w2v = pd.DataFrame(wordvec_arrays_test)


<ipython-input-54-77d3b8c91b5c>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vec += model_w2v[word].reshape((1, size))


In [ ]:
# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_w2v, y_train)
pred = lr_clf.predict(X_test_w2v)
print('Word2Vec Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.510


---

## 1) TF_IDF : 기본

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.638


## 2) CounVector : 기본

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 train / test 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 테스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 테스트의 CountVectorizer Shape: (350, 1131)


## 3) Pipe Line : LogisticRegression

### (1) TF_IDF : GridSearch (X)

In [ ]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
pred_probs = pipeline.predict_proba(X_test)[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

NameError: ignored

### (2) TF-IDF : GridSearch (O)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700], 
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 1)} 0.4978549978549978
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.504


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### (3) CountVectorizer : GridSearch (X)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score


pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
pred_probs = pipeline.predict_proba(X_test)[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

## 5) Pipe Line : Randomforest

### (1) Countvetorzier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

vectorizer = CountVectorizer(analyzer = 'word',
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             min_df = 2,
                             ngram_range = (1,3),
                             max_features = 20000)



forest = RandomForestClassifier(
     n_jobs = -1, random_state = 0
)

pipeline = Pipeline([
    ('vect', vectorizer),
    ('rf_clf', forest)

])

params = { 'rf_clf__n_estimators' : [50,70,80,100],
           'rf_clf__max_depth' : [3, 6, 9],
           'vect__min_df' : [2,3,4],
           'vect__ngram_range' : [(1,1),(1,2),(1,3)]
          }

grid_cv_pipe_rf = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe_rf.fit(X_train , y_train)
print(grid_cv_pipe_rf.best_params_ , grid_cv_pipe_rf.best_score_)

pred = grid_cv_pipe_rf.predict(X_test)
print('Pipeline을 통한 RandomFOrest 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


Fitting 3 folds for each of 1944 candidates, totalling 5832 fits
{'rf_clf__max_depth': 3, 'rf_clf__max_features': 'auto', 'rf_clf__min_samples_leaf': 2, 'rf_clf__min_samples_split': 2, 'rf_clf__n_estimators': 70, 'vect__min_df': 3, 'vect__ngram_range': (1, 2)} 0.6143530798703213
Pipeline을 통한 RandomFOrest 의 예측 정확도는 0.584


### (2) TF-IDF


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

tf = TfidfVectorizer(stop_words='english')


forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state = 0
)

pipeline = Pipeline([
    ('tf_idf', tf),
    ('rf_clf', forest)

])

params = { 'rf_clf__n_estimators' : [50,70,80, 100],
           'rf_clf__max_depth' : [3, 6, 9],
           'tf_idf__ngram_range': [(1,1), (1,2), (1,3)],
           'tf_idf__max_df': [100, 300, 500],
          }

grid_cv_pipe_rf_2 = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe_rf_2.fit(X_train , y_train)
print(grid_cv_pipe_rf_2.best_params_ , grid_cv_pipe_rf_2.best_score_)

pred = grid_cv_pipe_rf_2.predict(X_test)
print('Pipeline을 통한 RandomForest 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


# 4. 비지도 학습

In [ ]:
# X_test 를 사용하여 비지도 학습에 사용할 X_test_new 만들기
X_test_un = []
for i in range(len(X_test)):
  empty = []
  empty.append(X_test[i])
  X_test_un.append(empty)

X_test_un = pd.DataFrame(X_test_un)

vader_preds = np.zeros(len(X_test_un))
vader_preds = pd.DataFrame(vader_preds)
X_test_new = pd.concat([vader_preds, X_test_un], axis = 1)
X_test_new.columns=['vader_preds','text']
X_test_new

X_test_new

,vader_preds,text
0,0.0,atus oh way wish stay togeth forev
1,0.0,realli sad troubl stuff cri sigh cri lot multi...
2,0.0,atus work fine
3,0.0,plan drill outsid tonight fail drill light
4,0.0,enjoy wisdom alexandria toward
...,...,...
144,0.0,atus least andquothapi endingandquot
145,0.0,atus may see done militarymen
146,0.0,atus oh multistop probabl noah like boat cut m...
147,0.0,am best mood wish peopl talk right


In [ ]:
# 비지도 학습 함수 생성

import nltk
nltk.download('all')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# 감성 분석 함수 
def vader_polarity(text, threshold=0.1): # 임계값 0.1 로, 이거 우리가 조정가능
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(text) 

  # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
  agg_score = scores['compound']
  final_sentiment = 4 if agg_score >= threshold else 0
  return final_sentiment

In [ ]:
# 비지도 학습 실행

# apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 'vader_preds'에 저장
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

X_test_new['vader_preds'] = X_test_new['text'].apply(lambda x : vader_polarity(x, 0.1))
y_target = y_test.values
vader_preds = X_test_new['vader_preds'].values

print("정확도:", np.round(accuracy_score(y_target, vader_preds), 4))

정확도: 0.5705
